In [1]:
cd editGAN/

/root/editGAN


In [ ]:
!python align_mask_new.py -layer_sindex=11 -num_layer=20 ;shutdown

preprocessing semantic mask data
100%|█████████████████████████████████████████| 225/225 [00:19<00:00, 11.66it/s]
preprocessing done,semantic_masks_all.shape is (900, 12, 32, 32)
layer index:  11
 26%|██████████▌                              | 231/900 [08:05<22:55,  2.06s/it]

In [3]:
!pip install pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 11.7 MB 21.0 MB/s eta 0:00:01


In [6]:
!python semantic_channel_new.py

len(all_var_grad) 1
all_var_grad[0].shape (900, 512, 12, 3)
Layer 0
100%|████████████████████████████████████████| 512/512 [00:01<00:00, 316.13it/s]
all_var_grad[0][:,0,:,2][3] [331. 404. 248.   8.   6.  27.  nan  nan  nan  nan  nan  nan]
mask_counts2[3] [ True  True  True  True  True  True False False False False False False]
mask_counts3 [900 859 881 733 519 882 263  70  77  21 104  62]
mask_counts3 [900 859 881 733 519 882 263  70  77  21 104  62]


In [3]:
import numpy as np
scores=np.load('../autodl-tmp/editGAN/scores/semantic_top_32.npy',allow_pickle=True)
scores_c=np.concatenate(scores)
print(scores_c.shape)

target_index=(2,)
top_sum=scores_c[:,target_index].sum(axis=1)

tmp=list(np.arange(12))
for i in target_index:
    tmp.remove(i)
tmp=scores_c[:,tmp]
second_max=tmp.max(axis=1)
#print(top_sum)
#print(second_max)
select1=top_sum>0.5
print(select1.sum())
select2=top_sum-second_max>0.25
print(select2.sum())

select=np.logical_and(select1,select2)
print(select)
findex=np.arange(len(select))[select]
print(findex)

(4928, 12)
284
152
[False False False ... False False False]
[   7   21   37   51  173  188  235  307  309  337  398  400  403  424
  482  488  499  502  516  525  574  591  623  626  658  702  725  782
  797  847  870  880  914  977  999 1019 1027 1069 1071 1130 1156 1164
 1165 1199 1214 1225 1226 1239 1247 1251 1280 1322 1324 1337 1338 1373
 1378 1400 1491 1568 1590 1607 1612 1654 1657 1660 1729 1743 1754 1806
 1809 1860 1868 1891 1961 1963 1976 1981 2003 2015 2019 2051 2073 2075
 2125 2134 2150 2156 2218 2220 2233 2254 2284 2296 2336 2360 2364 2365
 2456 2531 2568 2630 2717 2732 2795 2916 2954 2987 2991 2997 3025 3090
 3091 3093 3130 3161 3179 3194 3218 3313 3328 3334 3355 3392 3411 3467
 3532 3541 3546 3551 3692 3942 3997 4033 4120 4135 4140 4210 4225 4284]


In [2]:
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
import time
import os
import copy
import torch
import dnnlib
import legacy
import time
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

In [5]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [5]:
pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu110/torch-1.7.0%2Bcu110-cp38-cp38-linux_x86_64.whl (1137.1 MB)
  Using cached https://download.pytorch.org/whl/cu110/torchvision-0.8.1%2Bcu110-cp38-cp38-linux_x86_64.whl (12.9 MB)
     |████████████████████████████████| 7.6 MB 266 kB/s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1+cu110
    Uninstalling torch-1.7.1+cu110:
      Successfully uninstalled torch-1.7.1+cu110
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.2+cu110
    Uninstalling torchvision-0.8.2+cu110:
      Successfully uninstalled torchvision-0.8.2+cu110
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.7.2
    Uninstalling torchaudio-0.7.2:
      Successfully uninstalled torchaudio-0.7.2


In [3]:
#导入新版网络
device = torch.device('cuda')
with dnnlib.util.open_url("cloth-v2-620t-s-test.pkl") as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)

In [4]:
#test channel effect
import os
import numpy as np
from PIL import Image

layer=9
channel=122

name = '2.npy'
data = np.load(os.path.join('../autodl-tmp/editGAN/latent_S', name), allow_pickle=True)
ss = []
ss_pos=[]
ss_neg=[]
for i in range(20):
    ss.append(data[i].cuda())
    ss_pos.append(ss[i].clone())
    ss_neg.append(ss[i].clone())

print('ss',ss[layer][0][channel])

ss_pos[layer][0][channel]+=100.
print("ss_pos",ss_pos[layer][0][channel])
ss_neg[layer][0][channel]-=100.
print("ss_neg",ss_neg[layer][0][channel])
print("ss",ss[layer][0][channel])

output_orig,_,_=G.synthesis(None,return_style=True,use_styles=True,input_styles=ss,noise_mode='const')
img1 = (output_orig.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img1[0].cpu().numpy(), 'RGB').save("../autodl-tmp/orig.jpg")

output_3,_,_=G.synthesis(None,return_style=True,use_styles=True,input_styles=ss_pos,noise_mode='const')
img2 = (output_3.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img2[0].cpu().numpy(), 'RGB').save("../autodl-tmp/l"+str(layer)+"c"+str(channel)+"_pos.jpg")

output_7,_,_=G.synthesis(None,return_style=True,use_styles=True,input_styles=ss_neg,noise_mode='const')
img3 = (output_7.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img3[0].cpu().numpy(), 'RGB').save("../autodl-tmp/l"+str(layer)+"c"+str(channel)+"_neg.jpg")
print("done")

ss tensor(-1.6622, device='cuda:0')
ss_pos tensor(98.3378, device='cuda:0')
ss_neg tensor(-101.6622, device='cuda:0')
ss tensor(-1.6622, device='cuda:0')
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
done


/root/editGAN/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/root/editGAN/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/root/editGAN/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/root/miniconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 969, in load
    return _jit_compile(
  File "/root/miniconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1189, in _jit_compile
    baton.wait()
  File "/root/miniconda3/lib/python3.8/site-packages/torch/utils/file_baton.py", line 48, in wait
    time.sleep(self.wait_seconds)
KeyboardInterrupt

  warnings.warn('Failed to build

In [ ]:
!python create_pics.py -pic_num=200 -sema_inx=6

[1049 1212 1224 1563 1585 1618 1635 1687 1927 1977 1998 2008 2052 2097
 2174 2205 2380 2461 2466 2476 2523 2603 2621 2737 2745 2775 2800 2839
 3002 3042 3118 3450 3462 3469 3580 3741 3951 3981 4349 4375 4399 4445
 4454 4669 4670]
calculating the layers index and channels index that effect the semantic most
l 3 c 25
l 3 c 188
l 3 c 200
l 5 c 27
l 5 c 49
l 5 c 82
l 5 c 99
l 5 c 151
l 5 c 391
l 5 c 441
l 5 c 462
l 5 c 472
l 6 c 4
l 6 c 49
l 6 c 126
l 6 c 157
l 6 c 332
l 6 c 413
l 6 c 418
l 6 c 428
l 6 c 475
l 8 c 43
l 8 c 61
l 8 c 177
l 8 c 185
l 8 c 215
l 8 c 240
l 8 c 279
l 8 c 442
l 8 c 482
l 9 c 46
l 9 c 378
l 9 c 390
l 9 c 397
l 9 c 508
l 11 c 157
l 11 c 367
l 11 c 397
l 12 c 253
l 14 c 23
l 14 c 47
l 14 c 93
l 14 c 102
l 15 c 61
l 15 c 62
  0%|                                                   | 0/200 [00:00<?, ?it/s]Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 